In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [3]:
dataset_path = '/content/Housing.csv'
df = pd.read_csv(dataset_path)

In [6]:
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [7]:
categorical_cols = df.select_dtypes(include=['object']).columns.to_list()
print(categorical_cols)

['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']


In [10]:
# Sử dụng OrdinalEncoder để chuyển đổi các cột này
ordinal_encoder = OrdinalEncoder()
encoded_categorical_cols = ordinal_encoder.fit_transform(df[categorical_cols])

# Chuyển kết quả từ mảng numpy thành DataFrame và gán lại tên cột
encoded_categorical_df = pd.DataFrame(encoded_categorical_cols, columns=categorical_cols)

# Bỏ các cột chuỗi gốc ra khỏi DataFrame ban đầu
numerical_df = df.drop(categorical_cols, axis=1)

# Gộp lại DataFrame với các cột đã được mã hóa
encoded_df = pd.concat([numerical_df, encoded_categorical_df], axis=1)

# Hiển thị kết quả dưới dạng bảng
print(encoded_df.head().to_string(index=False))

   price  area  bedrooms  bathrooms  stories  parking  mainroad  guestroom  basement  hotwaterheating  airconditioning  prefarea  furnishingstatus
13300000  7420         4          2        3        2       1.0        0.0       0.0              0.0              1.0       1.0               0.0
12250000  8960         4          4        4        3       1.0        0.0       0.0              0.0              1.0       0.0               0.0
12250000  9960         3          2        2        2       1.0        0.0       1.0              0.0              0.0       1.0               1.0
12215000  7500         4          2        2        3       1.0        0.0       1.0              0.0              1.0       1.0               0.0
11410000  7420         4          1        2        2       1.0        1.0       1.0              0.0              1.0       0.0               0.0


In [11]:
# Khởi tạo đối tượng StandardScaler
normalizer = StandardScaler()

# Chuẩn hóa dữ liệu
dataset_arr = normalizer.fit_transform(encoded_df)

# Hiển thị kết quả đã chuẩn hóa
print(dataset_arr[:5])

[[ 4.56636513  1.04672629  1.40341936  1.42181174  1.37821692  1.51769249
   0.40562287 -0.46531479 -0.73453933 -0.2192645   1.4726183   1.80494113
  -1.40628573]
 [ 4.00448405  1.75700953  1.40341936  5.40580863  2.53202371  2.67940935
   0.40562287 -0.46531479 -0.73453933 -0.2192645   1.4726183  -0.55403469
  -1.40628573]
 [ 4.00448405  2.21823241  0.04727831  1.42181174  0.22441013  1.51769249
   0.40562287 -0.46531479  1.3613975  -0.2192645  -0.67906259  1.80494113
  -0.09166185]
 [ 3.98575468  1.08362412  1.40341936  1.42181174  0.22441013  2.67940935
   0.40562287 -0.46531479  1.3613975  -0.2192645   1.4726183   1.80494113
  -1.40628573]
 [ 3.55497918  1.04672629  1.40341936 -0.57018671  0.22441013  1.51769249
   0.40562287  2.14908276  1.3613975  -0.2192645   1.4726183  -0.55403469
  -1.40628573]]


In [12]:
# Tách dữ liệu thành X và y
X, y = dataset_arr[:, 1:], dataset_arr[:, 0]

# Hiển thị một phần dữ liệu để kiểm tra
print("X (features):\n", X[:5])
print("y (target):\n", y[:5])

X (features):
 [[ 1.04672629  1.40341936  1.42181174  1.37821692  1.51769249  0.40562287
  -0.46531479 -0.73453933 -0.2192645   1.4726183   1.80494113 -1.40628573]
 [ 1.75700953  1.40341936  5.40580863  2.53202371  2.67940935  0.40562287
  -0.46531479 -0.73453933 -0.2192645   1.4726183  -0.55403469 -1.40628573]
 [ 2.21823241  0.04727831  1.42181174  0.22441013  1.51769249  0.40562287
  -0.46531479  1.3613975  -0.2192645  -0.67906259  1.80494113 -0.09166185]
 [ 1.08362412  1.40341936  1.42181174  0.22441013  2.67940935  0.40562287
  -0.46531479  1.3613975  -0.2192645   1.4726183   1.80494113 -1.40628573]
 [ 1.04672629  1.40341936 -0.57018671  0.22441013  1.51769249  0.40562287
   2.14908276  1.3613975  -0.2192645   1.4726183  -0.55403469 -1.40628573]]
y (target):
 [4.56636513 4.00448405 4.00448405 3.98575468 3.55497918]


In [13]:
# Thiết lập tham số chia dữ liệu
test_size = 0.3
random_state = 1
is_shuffle = True

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=test_size,
    random_state=random_state,
    shuffle=is_shuffle
)

# Hiển thị kích thước của các tập dữ liệu để kiểm tra
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)

X_train shape: (381, 12)
X_val shape: (164, 12)
y_train shape: (381,)
y_val shape: (164,)


In [14]:
# Khởi tạo mô hình Random Forest
regressor_rf = RandomForestRegressor(random_state=random_state)

# Huấn luyện mô hình
regressor_rf.fit(X_train, y_train)

RandomForestRegressor(random_state=1)

In [16]:
from sklearn.ensemble import AdaBoostRegressor

# Khởi tạo mô hình AdaBoost
regressor_ab = AdaBoostRegressor(random_state=random_state)

# Huấn luyện mô hình
regressor_ab.fit(X_train, y_train)

AdaBoostRegressor(random_state=1)

In [17]:
from sklearn.ensemble import GradientBoostingRegressor

# Khởi tạo mô hình Gradient Boosting
regressor_gb = GradientBoostingRegressor(random_state=random_state)

# Huấn luyện mô hình
regressor_gb.fit(X_train, y_train)

GradientBoostingRegressor(random_state=1)

In [19]:
# Sử dụng mô hình để dự đoán trên tập kiểm tra
y_pred = regressor_rf.predict(X_val)

# Tính toán MAE và MSE
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)

# Hiển thị kết quả đánh giá
print('Evaluation results on validation set:')
print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')

Evaluation results on validation set:
Mean Absolute Error: 0.46093873321571177
Mean Squared Error: 0.37944418523089524
